In [18]:
# Prepare features and all targets
features = df.drop(targets, axis=1)
Y = df[targets]

# Single split for all targets
X_train, X_test, Y_train, Y_test = train_test_split(
    features, Y, stratify=Y['Machine failure'], test_size=0.2, random_state=42
)

models = {}
for target in targets:
    y_train = Y_train[target]
    y_test = Y_test[target]
    clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    clf.fit(X_train, y_train)
    models[target] = clf

# Store predictions and actuals in results DataFrame
X_test_df = X_test.copy()
for target in targets:
    X_test_df[f'Predicted_{target}'] = models[target].predict(X_test)
    X_test_df[f'Actual_{target}'] = Y_test[target].values

# Write results to SQL Server
engine = create_engine('mssql+pyodbc://localhost/Project1?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes')
X_test_df.to_sql('ml_predictions', engine, if_exists='replace', index=False)


28

In [20]:
import joblib

# Train your model as shown before...
joblib.dump(clf, 'machine_failure_model.pkl')


['machine_failure_model.pkl']